In [1]:
import os
import librosa
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, RepeatVector
from sklearn.preprocessing import MinMaxScaler


2023-12-15 08:41:30.431971: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 08:41:30.433448: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 08:41:30.462840: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 08:41:30.463438: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 08:41:31.099444: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
# Loading the audio files.
def load_and_preprocess_audio(directory, sr=44100, duration=60):
    max_length = sr * duration
    audio_data = []
    for file in os.listdir(directory):
        if file.endswith('.wav'):
            path = os.path.join(directory, file)
            data, _ = librosa.load(path, sr=sr)
            if len(data) > max_length:
                data = data[:max_length]
            else:
                data = np.pad(data, (0, max_length - len(data)), 'constant')
            audio_data.append(data)
    return np.array(audio_data)


In [3]:
# Extracting the features from the audio data.
#TODO : Look for better feature extractor.
def extract_features(audio_data):
    features = [librosa.feature.mfcc(y=data, sr=22050, n_mfcc=20).T for data in audio_data]
    return features


In [4]:
# preprocessing the data using MinMaxScaler.
def preprocess_data(features):
    scaler = MinMaxScaler()
    scaled_features = [scaler.fit_transform(feature) for feature in features]
    return np.array(scaled_features)


In [5]:
# LSTM Model
def build_model(input_shape):
    model = Sequential([
        LSTM(128, input_shape=input_shape, return_sequences=True),
        LSTM(64, return_sequences=False),
        RepeatVector(input_shape[0]),
        LSTM(64, return_sequences=True),
        LSTM(128, return_sequences=True),
        TimeDistributed(Dense(input_shape[1]))
    ])
    model.compile(optimizer='adam', loss='mse')
    return model


In [6]:
# Training the model.   
def train_model(model, data, epochs=10, batch_size=32):
    model.fit(data, data, epochs=epochs, batch_size=batch_size)


In [7]:
# Detecting the anamolies.
def detect_anomalies(model, data, threshold=0.1):
    predictions = model.predict(data)
    anomalies = []
    for i in range(len(data)):
        loss = np.mean(np.abs(data[i] - predictions[i]))
        if loss > threshold:
            anomalies.append(i)
    return anomalies

        

In [8]:
directory = 'Audio_Work/birth_inference_minutes_wav/output_2023-10-09_06-33-01'
audio_data = load_and_preprocess_audio(directory)
features = extract_features(audio_data)
processed_data = preprocess_data(features)

model = build_model(input_shape=processed_data.shape[1:])
train_model(model, processed_data)

anomalies = detect_anomalies(model, processed_data)
print("Anomalies found at indices:", anomalies)


2023-12-15 08:41:34.745748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 08:41:34.746624: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
1/1 [==============================] - 13s 13s/step - loss: 0.2889
Epoch 2/10
1/1 [==============================] - 8s 8s/step - loss: 0.2317
Epoch 3/10
1/1 [==============================] - 9s 9s/step - loss: 0.1686
Epoch 4/10
1/1 [==============================] - 9s 9s/step - loss: 0.0907
Epoch 5/10
1/1 [==============================] - 9s 9s/step - loss: 0.0436
Epoch 6/10
1/1 [==============================] - 8s 8s/step - loss: 0.0480
Epoch 7/10
1/1 [==============================] - 8s 8s/step - loss: 0.0389
Epoch 8/10
1/1 [==============================] - 7s 7s/step - loss: 0.0281
Epoch 9/10
1/1 [==============================] - 7s 7s/step - loss: 0.0259
Epoch 10/10
1/1 [==============================] - 2s 2s/step
Anomalies found at indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
